Pipeline de dados do Telegram:

**Por meio de chatbot que é:** Um chatbot é um tipo de software que interage com usuários através de conversas automatizadas em plataformas de mensagens. Uma aplicação comum de chatbots é o seu uso no atendimento ao cliente, onde, de maneira geral, ajudam clientes a resolver problemas ou esclarecer dúvidas recorrentes antes mesmo que um atendente humano seja acionado.

**Telegram que é:** Uma plataforma de mensagens instantâneas freeware (distribuído gratuitamente) e, em sua maioria, open source. É muito popular entre desenvolvedores por ser pioneiro na implantação da funcionalidade de criação de chatbots, que, por sua vez, permitem a criação de diversas automações.

**Com o objetivo de:** de realizar a análise exploratória de dados enviadas a um chatbot para responder perguntas como: 
1. Qual o horário que os usuários mais acionam o bot? 
2. Qual o problema ou dúvida mais frequente? 
3. O bot está conseguindo resolver os problemas ou esclarecer as dúvidas? 
4. Etc.

Para isso, vamos construir um pipeline de dados que ingira, processe, armazene e exponha mensagens de um grupo do Telegram para que profissionais de dados possam realizar análises. A arquitetura proposta é dividida em duas: transacional, no Telegram, onde os dados são produzidos, e analítica, na Amazon Web Services (AWS), onde os dados são analisados.


![imagem](https://github.com/NeimarSntn/Imagens/blob/main/Profissao%20Analista%20de%20dados%20M42%20Material%20de%20apoio%20arch.png?raw=true)

O Telegram representa a fonte de dados transacionais. Mensagens enviadas por usuários em um grupo são capturadas por um bot e redirecionadas via webhook do backend do aplicativo para um endpoint (endereço web que aceita requisições HTTP) exposto pelo AWS API Gateway. As mensagens trafegam no corpo ou payload da requisição.
**AWS API Gateway**
![imagem](https://github.com/NeimarSntn/Imagens/blob/main/api.jpg?raw=true)

AWS | Ingestão 

Uma requisição HTTP com o conteúdo da mensagem em seu payload é recebida pelo AWS API Gateway que, por sua vez, as redireciona para o AWS Lambda, servindo assim como seu gatilho. Já o AWS Lambda recebe o payload da requisição em seu parâmetro event, salva o conteúdo em um arquivo no formato JSON (original, mesmo que o payload) e o armazena no AWS S3 particionado por dia.

**Lambda**
![imagem](https://github.com/NeimarSntn/Imagens/blob/main/lambda.jpg?raw=true)



AWS | ETL 

Uma vez ao dia, o AWS Event Bridge aciona o AWS Lambda que processa todas as mensagens do dia anterior (atraso de um dia ou D-1), denormaliza o dado semi-estruturado típico de arquivos no formato JSON, salva o conteúdo processado em um arquivo no formato Apache Parquet e o armazena no AWS S3 particionado por dia.

**AWS Event Bridge**
![imagem](https://github.com/NeimarSntn/Imagens/blob/main/bridge.jpg?raw=true)

**S3**
![imagem](https://github.com/NeimarSntn/Imagens/blob/main/s3.jpg?raw=true)

AWS | Apresentação 

Por fim, uma tabela do AWS Athena é apontada para o bucket do AWS S3 que armazena o dado processado: denormalizado, particionado e orientado a coluna. Profissionais de dados podem então executar consultas analíticas (agregações, ordenações, etc.) na tabela utilizando o SQL para a extração de insights.

**Quantidade de mensagens por dia.**

![imagem](https://github.com/NeimarSntn/Imagens/blob/main/quantidade%20por%20dia.jpg?raw=true)

**Mensagens por usuário.**

![imagem](https://github.com/NeimarSntn/Imagens/blob/main/mensagem%20por%20usuraio.jpg?raw=true)

**Tamanho mensagem.**

![imagem](https://github.com/NeimarSntn/Imagens/blob/main/tamanho%20mensagem.jpg?raw=true)

**Horário mensagem.**

![imagem](https://github.com/NeimarSntn/Imagens/blob/main/hora%20mensagem.jpg?raw=true)

